In [1]:
# import libraries
import pandas as pd 
import numpy as np 
import sqlite3
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [3]:
# Extract

# Load the dataset
csv_path = "../data/Online Retail.csv"  
df = pd.read_csv(csv_path, encoding='ISO-8859-1')
logging.info(f"Loaded {len(df)} rows from {csv_path}")


# Handle missing values
df = df.dropna(subset=['CustomerID', 'InvoiceDate', 'Quantity', 'UnitPrice', 'Country', 'Description', 'InvoiceNo', 'StockCode'])
logging.info(f"Rows after dropping missing CustomerID/InvoiceDate: {len(df)}")

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df = df.dropna(subset=['InvoiceDate'])
logging.info(f"Rows after ensuring valid InvoiceDate: {len(df)}")

# Convert Quantity and UnitPrice to numeric
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')
df.head()


INFO: Loaded 541909 rows from ../data/Online Retail.csv
INFO: Rows after dropping missing CustomerID/InvoiceDate: 406829
INFO: Rows after ensuring valid InvoiceDate: 406829


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# Transform 

# Remove outliers: Quantity < 0 or UnitPrice <= 0
df = df[(df['Quantity'] >= 0) & (df['UnitPrice'] > 0)]
logging.info(f"Rows after removing outliers: {len(df)}")

# Calculate TotalSales
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

# Filter for sales in the last year (Aug 12, 2024 to Aug 12, 2025)
cutoff_date = pd.Timestamp('2024-08-12')
end_date = pd.Timestamp('2025-08-12')
df_last_year = df[(df['InvoiceDate'] >= cutoff_date) & (df['InvoiceDate'] <= end_date)]
logging.info(f"Rows for sales in the last year: {len(df_last_year)}")

# Customer summary: total purchases and country
customer_summary = df_last_year.groupby('CustomerID').agg(
    TotalPurchases=('TotalSales', 'sum'),
    Country=('Country', 'first')
).reset_index()

logging.info(f"Customer summary rows: {len(customer_summary)}")

# Preview transformed data
df.head()

INFO: Rows after removing outliers: 397884
INFO: Rows for sales in the last year: 0
INFO: Customer summary rows: 0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [14]:

    # Set up logging
    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

    # Extract
    df = pd.read_csv(csv_path, encoding='ISO-8859-1')
    logging.info(f"Extracted {len(df)} rows from {csv_path}")

    df = df.dropna(subset=['CustomerID', 'InvoiceDate', 'Quantity', 'UnitPrice', 'Country', 'Description', 'InvoiceNo', 'StockCode'])
    logging.info(f"Rows after dropping missing values: {len(df)}")

    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
    df = df.dropna(subset=['InvoiceDate'])
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
    df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')
    logging.info(f"Rows after type conversions: {len(df)}")

    # Transform
    df = df[(df['Quantity'] >= 0) & (df['UnitPrice'] > 0)]
    logging.info(f"Rows after removing outliers: {len(df)}")

    df['TotalSales'] = df['Quantity'] * df['UnitPrice']

    cutoff_date = pd.Timestamp('2024-08-12')
    end_date = pd.Timestamp('2025-08-12')
    df_last_year = df[(df['InvoiceDate'] >= cutoff_date) & (df['InvoiceDate'] <= end_date)]
    logging.info(f"Rows for sales in the last year: {len(df_last_year)}")

    # CustomerDim
    customer_dim = df_last_year[['CustomerID', 'Country']].drop_duplicates().copy()
    customer_dim['Name'] = "Unknown"
    customer_dim['Gender'] = "Unknown"
    customer_dim['Email'] = "unknown@example.com"
    customer_dim['CustomerKey'] = customer_dim.index + 1
    customer_dim = customer_dim[['CustomerKey', 'Name', 'Country', 'Gender', 'Email', 'CustomerID']]
    customer_dim.rename(columns={'Country': 'Location'}, inplace=True)
    logging.info(f"CustomerDim rows: {len(customer_dim)}")

    # TimeDim
    time_dim = df_last_year[['InvoiceDate']].drop_duplicates().copy()
    time_dim['TimeKey'] = time_dim.index + 1
    time_dim['Date'] = time_dim['InvoiceDate'].dt.date
    time_dim['Day'] = time_dim['InvoiceDate'].dt.day
    time_dim['Month'] = time_dim['InvoiceDate'].dt.month
    time_dim['Quarter'] = time_dim['InvoiceDate'].dt.quarter
    time_dim['Year'] = time_dim['InvoiceDate'].dt.year
    time_dim = time_dim[['TimeKey', 'Date', 'Day', 'Month', 'Quarter', 'Year', 'InvoiceDate']]
    logging.info(f"TimeDim rows: {len(time_dim)}")

    # ProductDim
    product_dim = df_last_year[['StockCode', 'Description', 'UnitPrice']].drop_duplicates().copy()
    product_dim['ProductKey'] = product_dim.index + 1
    product_dim['Name'] = product_dim['Description']
    product_dim['Category'] = "General"
    product_dim = product_dim[['ProductKey', 'Name', 'Category', 'UnitPrice', 'Description', 'StockCode']]
    logging.info(f"ProductDim rows: {len(product_dim)}")

    # Prepare for SalesFact
    df_last_year['Date'] = df_last_year['InvoiceDate'].dt.date

    # Merge keys
    sales_fact = df_last_year.merge(customer_dim[['CustomerKey', 'CustomerID']], on='CustomerID', how='left')
    sales_fact = sales_fact.merge(time_dim[['TimeKey', 'Date']], on='Date', how='left')
    sales_fact = sales_fact.merge(product_dim[['ProductKey', 'Description']], on='Description', how='left')
    sales_fact = sales_fact[['ProductKey', 'CustomerKey', 'TimeKey', 'TotalSales', 'Quantity']]
    sales_fact.rename(columns={'TotalSales': 'SalesAmount'}, inplace=True)
    logging.info(f"SalesFact rows: {len(sales_fact)}")

    # Load
    db_path = "../data/retail_dw.db"

    conn = sqlite3.connect(db_path)
    customer_dim.to_sql('CustomerDim', conn, if_exists='replace', index=False)
    time_dim.to_sql('TimeDim', conn, if_exists='replace', index=False)
    product_dim.to_sql('ProductDim', conn, if_exists='replace', index=False)
    sales_fact.to_sql('SalesFact', conn, if_exists='replace', index=False)
    conn.close()
    logging.info("ETL process completed and data loaded into database.")

INFO: Extracted 541909 rows from ../data/Online Retail.csv
INFO: Rows after dropping missing values: 406829
INFO: Rows after type conversions: 406829
INFO: Rows after removing outliers: 397884
INFO: Rows for sales in the last year: 0
INFO: CustomerDim rows: 0
INFO: TimeDim rows: 0
INFO: ProductDim rows: 0
INFO: SalesFact rows: 0
INFO: ETL process completed and data loaded into database.
